In [57]:
import numpy as np
import pandas as pd
import re
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [58]:
# import nltk
# nltk.download('stopwords')

In [59]:
matrix = pd.read_parquet('../datasets/processed/frequency_matrix_2013.parquet')

In [60]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [61]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [62]:
resumo = 'Este trabalho foi desenvolvido utilizando as tecnologias Python, Pandas, Numpy, NLTK, Unidecode e LancasterStemmer. O objetivo é realizar a limpeza de um texto, removendo stopwords, pontuações e acentuações, além de realizar a redução de palavras para sua forma raiz. O texto utilizado para teste foi o resumo deste trabalho. nitrosilo complexo rutenio'

In [63]:
resumo = resumo.lower()

In [64]:
resumo = re.sub(r'[^a-zA-Z0-9\s]+', '', resumo)

In [65]:
resumo = remove_stopwords(resumo)
resumo = lc_stem(resumo)
resumo = unidecode(resumo)

del remove_stopwords, lc_stem

In [66]:
resumo

'trabalho desenvolvido utilizando tecnologia python panda numpy nltk unidecod lancasterstem objetivo realiz limpez texto removendo stopword pontua acentua alm realiz reduo palavra form raiz texto utilizado test resumo dest trabalho nitrosilo complexo rutenio'

In [67]:
def verify_concat(i: int, item: str, text: str) -> str:
    elements = wordpunct_tokenize(text)[i:]
    
    if len(elements) > 1:
        test_1 = item + ' ' + elements[1]
        test_2 = item  + elements[1]
    else:
        return i, item
    
    if test_1 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_1, text)
    if test_2 in matrix.palavra_chave.values:
        return verify_concat(i+1, test_2, text)
    else:
        return i, item

def get_keywords(text: str) -> list:
    result = []
    i = 0
    elements = wordpunct_tokenize(text)
    while i < len(elements):
        item = elements[i]
        if item in matrix.palavra_chave.values:
            i, item = verify_concat(i, item, text)
            result.append(item)
        i+=1
    return result

In [68]:
result = list(set(get_keywords(resumo)))
result

['texto',
 'utilizando',
 'nltk',
 'test',
 'nitrosilo complexo rutenio',
 'limpez',
 'palavra',
 'raiz',
 'python',
 'objetivo',
 'alm',
 'tecnologia',
 'resumo',
 'form',
 'trabalho']

In [69]:
# matrix[matrix.palavra_chave.str.fullmatch(result[0])].sort_values(by='frequencia', ascending=False)
matrix[matrix.palavra_chave.str.fullmatch(result[0])].frequencia.sum()

20023

In [70]:
frequency_general = matrix.frequencia.sum()

for item in result:
    this_frequency = matrix[matrix.palavra_chave.str.fullmatch(item)].frequencia.sum()
    print(f'{item}: {(this_frequency/frequency_general) * 100}')

texto: 0.04344467049019401
utilizando: 0.02659665239318774
nltk: 4.339476650870899e-06
test: 0.06401812929197295
nitrosilo complexo rutenio: 3.6885551532402644e-05
limpez: 0.0010544928261616284
palavra: 0.006162056844236677
raiz: 0.002759907149953892
python: 3.4715813206967194e-05
objetivo: 0.09714786378304682
alm: 0.055256725933864596
tecnologia: 0.007871810644679811
resumo: 0.0027946229631608593
form: 0.2398927784751196
trabalho: 0.12835737985611034


In [72]:
subarea_groupped = matrix.groupby('subarea').frequencia.sum().sort_values(ascending=False).reset_index()

for item in result:
    words_frequency = matrix[matrix.palavra_chave.str.fullmatch(item)].sort_values(by='frequencia', ascending=False).loc[:, ['subarea', 'frequencia']].copy()
    matrix_corr = words_frequency.join(subarea_groupped.set_index('subarea'), on='subarea', how='left', lsuffix='_this_word', rsuffix='_all_groupped')
    matrix_corr['corr'] = (matrix_corr.frequencia_this_word / matrix_corr.frequencia_all_groupped) * 100
    corr = matrix_corr.sort_values(by='corr', ascending=False).reset_index(drop=True).loc[0, ['subarea', 'corr']]
    print(f'{item}: {corr[0]} - {corr[1]}')

texto: linguística e literatura - 0.22056152369090248
utilizando: química - 0.07176777644514405
nltk: linguística e literatura - 0.00010591189613008524
test: educação física - 0.22978591219598782
nitrosilo complexo rutenio: ciências ambientais - 0.0005564107795315021
limpez: geografia - 0.007642830421683113
palavra: linguística e literatura - 0.02022917216084628
raiz: ciências agrárias i - 0.019929185401145796
python: ciência da computação - 0.0004810994083680026
objetivo: educação física - 0.13770513627677483
alm: ciências biológicas ii - 0.09491128115669825
tecnologia: ensino - 0.035947116656096505
resumo: zootecnia / recursos pesqueiros - 0.013288488041870817
form: comunicação e informação - 0.538162865537988
trabalho: matemática / probabilidade e estatística - 0.34520993163137503
